In [1]:
from IPython.display import clear_output
from music21 import *
import numpy as np
import os

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
#test_path = "../dataset/combined/alb_esp1.mid"
test_path = "../dataset/combined/DEB_CLAI.mid"

In [1]:
# Function to open a midi file for further manipulation
# Params: string path to a midi file
def open_midi_file(midi_file_path):
    midifile = converter.parse(midi_file_path)
    return midifile

# Function to open a midi file's sheet variant in MuseScore, requires MuseScore to be installed
# PS: If at first it doesn't work, try following http://web.mit.edu/music21/doc/installing/installWindows.html#install-music21
# If it still doesn't work, try running the function a second time for the heck of it. Environment variables be crazy.
# Params: string path to a midi file
def open_midi_file_musescore(midi_file_path):
    midi_file = open_midi_file(midi_file_path)
    midi_file.show()
    
# Function to play a midi file
# Params: string path to a midi file
def play_midi_file(midi_file_path):
    midi_file = open_midi_file(midi_file_path)
    midi_file.show("midi")

# Function to transpose a midi stream to C major if the song was originally in a major key, or A minor if it was in minor
# Params: midi stream (midi file opened with open_midi_file())
def transpose_midi_key(midi_file):
    midi_key = midi_file.analyze('key')
    if (midi_key.mode == "major"):
        get_interval = interval.Interval(k.tonic, pitch.Pitch('C'))
    if (midi_key.mode == "minor"):
        get_interval = interval.Interval(k.tonic, pitch.Pitch('A'))
    transposed_midi = midi_file.transpose(get_interval)
    return transposed_midi

# Function to convert the whole dataset to tokens and save them in a single .txt file
# If you're running this for the first time, it will take a while (more than an hour) because music21 converter.parse
# is quite slow. Subsequent times are fast, however, as the parser has pickled the midis somewhere.
# Params: string path to folder containing midis, string name of the output file 
def write_midis_to_txt_as_tokens(midi_folder_path, output_file_name, transpose_bool = False):
    
    def print_progress(progress, number_of_files, number_of_tokens):
        clear_output(wait=True)
        print("{0} / {1} generated. Number of tokens in current midi: {2}".format(progress, number_of_files, number_of_tokens))
    
    path = midi_folder_path
    filenames = os.listdir(path)

    with open(output_file_name, "w") as txt:
        progress = 1
        for filename in filenames:
            midi_as_tokens = convert_midi_to_tokens(path + filename, transpose_bool)
            print_progress(progress, len(filenames), len(midi_as_tokens))
            txt.write(' '.join(midi_as_tokens) + '\n')
            progress += 1

# Function to print the list of unique token values from a list of tokens
# Params: a list of tokens, string type of token (wait, note, stop_note) 
def get_unique_tokens(tokens, token_type):
    return set([i.split(":")[1] for i in tokens if i.startswith(token_type)])
    
# Function to make sure that a list of tokens contains a stop_note event for every note starting event
# Works similarly to adding a note in the tokens to midi conversion function
# Params: a list of tokens
# Returns true if all notes end properly, false if not
def validate_tokens(tokens):
    
    # Keeps track of the current token
    current_token_index = 0
    
    for token in tokens:
        token_type = token.split(":")[0]
        token_value = token.split(":")[1]
        
        if token_type == "note":
            found_corresponding_end_note = False
            note_midi_pitch = token_value
            
            for following_token in tokens[current_token_index + 1:]:
                following_token_type = following_token.split(":")[0]
                following_token_value = following_token.split(":")[1]
                    
                if following_token_type == "stop_note":
                    stopped_note_pitch = following_token_value
                    if (note_midi_pitch == stopped_note_pitch):
                        found_corresponding_end_note = True
                        break
            
            if not found_corresponding_end_note:
                return False
                
    return True
    
# Function to turn a midi file into text tokens
# Params: string path to a midi file, bool whether to transpose all midis or not
def convert_midi_to_tokens(midi_file_path, transpose_bool = False):
    
    # Function to remap velocity to not oversaturate the possible range of velocities
    # Completely made up values
    def remap_velocity(velocity):
        if velocity <= 42:
            return 40
        if velocity <= 84:
            return 70
        return 100
    
    def remap_tempo(tempo):
        if tempo <= 60:
            return 60
        if tempo >= 160:
            return 160
        return tempo - (tempo % 10)
            
    def velocity_change_handler(current_velocity, note_velocity, tokens_to_add):
        remapped_new_velocity = remap_velocity(note_velocity)
        if (current_velocity != remapped_new_velocity):
            current_velocity = remapped_new_velocity
            tokens_to_add.append("velocity:" + str(remapped_new_velocity))
        return tokens_to_add, current_velocity
    
    def tempo_change_handler(current_tempo, new_tempo, tokens_to_add):
        remapped_new_tempo = remap_tempo(new_tempo)
        if (current_tempo != remapped_new_tempo):
            current_tempo = remapped_new_tempo
            tokens_to_add.append("tempo:" + str(remapped_new_tempo))
        return tokens_to_add, current_tempo
    
    midi_file = open_midi_file(midi_file_path)
    
    if transpose_bool:
        midi_file = transpose_midi_key(midi_file)
    
    # A list to hold tokens
    tokens = list()
    
    # A list to keep track of which note events await a corresponding note off event
    notes_to_stop = list()
    
    # Keeps track of time since start of the midi piece
    current_offset = 0
    
    # Keeps track of velocity (note volume), used to add a velocity token every time this variable value changes
    # Theoretically can range from 0 to 127, but the possible range of values will be remapped to keep the final model simple
    # Default value of 70 is chosen because the author likes it
    current_velocity = 70
    
    # Keeps track of tempo, used to add a tempo token every time this variable value changes
    current_tempo = None
    
    # Keeps track of the offset of the previously handled midi event
    previous_offset = None
    
    # Because we are mushing different streams of MIDI events (e.g. left & right hand parts) into a single stream,
    # the tempos and time signatures are duplicated. The encoding, however, only needs to see them once.
    # previous_event is used to check if we just saw a tempo/timesig token to know if we should ignore its second occurrence.
    previous_event = None
    
    # Iterate over all midi events, sorted by offset (ascending)
    # and handle which tokens will be added to list of tokens
    for midi_event in midi_file.flat.elements:
        
        current_offset = round(float(midi_event.offset), 3)
        
        # At the end of the current loop, tokens in this list will be added to the final tokens list
        tokens_to_add = list()
        
        # Check if there are notes that should have ended between the last and current offset (included) 
        if len(notes_to_stop) != 0:
            for note_to_stop, when_to_stop in notes_to_stop.copy():
                if when_to_stop <= current_offset:
                    time_since_prev_offset = round(float(when_to_stop - previous_offset), 3)
                    if time_since_prev_offset > 0:
                        tokens_to_add.append("wait:" + str(time_since_prev_offset))
                        previous_offset = when_to_stop
                    tokens_to_add.append("stop_note:" + note_to_stop)
                    notes_to_stop.remove([note_to_stop, when_to_stop])
        
        # If the offset has changed by >0, account for it by adding a waiting token
        if (previous_offset != None and isinstance(midi_event, (note.Note, chord.Chord))):
            offset_change = round(float(current_offset - previous_offset), 3)
            if offset_change > 0:
                tokens_to_add.append("wait:" + str(offset_change))
        
        # Time signatures are unnecessary for playback, commented out.
        #if isinstance(midi_event, meter.TimeSignature) and not isinstance(previous_event, meter.TimeSignature):
        #    time_signature_value = str(midi_event.ratioString)
        #    tokens_to_add.append("timesignature:" + time_signature_value)
        
        if isinstance(midi_event, tempo.MetronomeMark) and not isinstance(previous_event, tempo.MetronomeMark):
            tempo_value = midi_event.number
            tokens_to_add, current_tempo = tempo_change_handler(current_tempo, tempo_value, tokens_to_add)
        
        # If the current midi event is a note, add a note token along with its midi pitch number
        # And remember when the note needs to be stopped
        if isinstance(midi_event, note.Note):
            midi_pitch = str(midi_event.pitch.midi)
            note_velocity = midi_event.volume.velocity
            
            tokens_to_add, current_velocity = velocity_change_handler(current_velocity, note_velocity, tokens_to_add)
            
            token_string = "note:" + midi_pitch
            note_end_offset = round(float(current_offset + midi_event.duration.quarterLength), 3)
            
            tokens_to_add.append(token_string)
            notes_to_stop.append([midi_pitch, note_end_offset])
        
        # If the current midi event is a chord, do the same as before for every note in the chord
        if isinstance(midi_event, chord.Chord):
            for individual_note in midi_event:
                midi_pitch = str(individual_note.pitch.midi)
                note_velocity = midi_event.volume.velocity
                
                tokens_to_add, current_velocity = velocity_change_handler(current_velocity, note_velocity, tokens_to_add)
                
                token_string = "note:" + midi_pitch
                note_end_offset = round(float(current_offset + individual_note.duration.quarterLength), 3)

                tokens_to_add.append(token_string)
                notes_to_stop.append([midi_pitch, note_end_offset])
        
        tokens.extend(tokens_to_add)
        previous_offset = current_offset
        previous_event = midi_event
    
    # After iterating through all midi events, it is necessary to check for note stopping events one more time,
    # since the last midi event could have been a note starting event
    tokens_to_add = list()
    if len(notes_to_stop) != 0:
        for note_to_stop, when_to_stop in notes_to_stop.copy():
            tokens_to_add.append("stop_note:" + note_to_stop)
            notes_to_stop.remove([note_to_stop, when_to_stop])
    tokens.extend(tokens_to_add)
    
    return tokens

# Function to convert list of text tokens to a Music21 midi stream
# Params: a list of tokens
def convert_tokens_to_midi(tokens):
    
    # A midi stream that will hold midi events converted from tokens
    midi_stream = stream.Stream()
    
    # Keeps track of the current token
    current_token_index = 0
    
    # Keeps track of offset
    current_offset = 0
    
    # Keeps track of velocity
    current_velocity = 70
    
    # The converter ignores BOS and EOS tags
    tokens = [token for token in tokens if token not in ["<EOS>", "<BOS>"]]
    
    for token in tokens:
        token_type = token.split(":")[0]
        token_value = token.split(":")[1]
        
        # Time signatures are unnecessary for playback. Commented out.
        #if token_type == "timesignature":
        #    timesignature_value = token_value
        #    midi_stream.append(meter.TimeSignature(timesignature_value))
        
        if token_type == "tempo":
            tempo_value = float(token_value)
            midi_stream.append(tempo.MetronomeMark(number=tempo_value))
            
        if token_type == "velocity":
            velocity_value = float(token_value)
            current_velocity = velocity_value
        
        # Converting a note-starting token to a midi event, we need to know its duration.
        # To find the duration, we look at the following tokens until we find a corresponding stop_note token.
        # While searching for the stop_note token, we add up the values of intermediate wait tokens, denoting duration.
        # We identify the corresponding stop_note by the note's midi pitch number. 
        if token_type == "note":
            note_duration = 0
            note_midi_pitch = token_value

            for following_token in tokens[current_token_index + 1:]:
                following_token_type = following_token.split(":")[0]
                following_token_value = following_token.split(":")[1]
                
                if following_token_type == "wait":
                    wait_duration = following_token_value
                    note_duration += float(wait_duration)
                    
                if following_token_type == "stop_note":
                    stopped_note_pitch = following_token_value
                    if (note_midi_pitch == stopped_note_pitch):
                        new_note = note.Note(int(note_midi_pitch))  
                        new_note.quarterLength = round(note_duration, 3)
                        new_note.volume.velocity = current_velocity
                        midi_stream.insert(current_offset, new_note)
                        break

        if token_type == "wait":
            wait_duration = token_value
            current_offset += float(wait_duration)

        current_token_index += 1

    return midi_stream

In [4]:
#midi_file = open_midi_file(test_path)

In [ ]:
#open_midi_file_musescore(test_path)

In [ ]:
#play_midi_file(test_path)

In [ ]:
#validate_tokens(convert_midi_to_tokens(test_path))

In [ ]:
#miditokens = convert_midi_to_tokens(test_path)

In [ ]:
#convert_tokens_to_midi(miditokens).show("midi")

In [ ]:
#get_unique_tokens(miditokens, "wait")

In [16]:
#write_midis_to_txt_as_tokens("../dataset/combined/", "./training_data.txt", False)
#write_midis_to_txt_as_tokens("../dataset/combined/", "./training_data_transposed.txt", True)

295 / 295 generated. Number of tokens in current midi: 21484
